In [1]:
import numpy as np
import pandas as pd
from math import *
import random

In [2]:
with open('malha1_re7500', 'w') as res, open('60_60_362_re7500_RANS_periodic') as f1,open('60_60_362_re7500_LES_periodic') as f2:
    for line1, line2 in zip(f1, f2):
        res.write("{} {}\n".format(line1.rstrip(), line2.rstrip()))

In [3]:
with open('malha1_re7500', "r") as f:
    linesf = random.sample(f.readlines(),1000)
for line in linesf:
    cleanedLine = line.strip()
    if cleanedLine: # is not empty
        file=np.loadtxt(linesf,comments="#")

In [4]:
SMALL=1e-20
Position=np.array((file[:,1],file[:,2],file[:,3])).transpose()
Vel=np.array((file[:,5],file[:,6],file[:,7])).transpose()
P=file[:,4]
GradVel=np.transpose(np.array([[file[:,17],file[:,18],file[:,19]],
                  [file[:,20],file[:,22],file[:,21]],
                  [file[:,23],file[:,24],file[:,25]]]),(2,0,1))
DivVel = np.array((file[:,17],file[:,22],file[:,25])).transpose()
GradP = np.array((file[:,26],file[:,27],file[:,28])).transpose()
HessP = np.transpose(np.array([[file[:,17],file[:,18],file[:,19]],
                [file[:,20],file[:,21],file[:,22]],
                [file[:,23],file[:,24],file[:,25]]]),(2,0,1))
ReStress=np.transpose(np.array([[file[:,8],file[:,11],file[:,13]],
                   [file[:,11],file[:,9],file[:,12]],
                   [file[:,13],file[:,12],file[:,10]]]),(2,0,1))
K=file[:,14]
Eps=file[:,15]
TurbVisc=file[:,16]
D=GradVel+np.transpose(GradVel,(0,2,1))
W=GradVel-np.transpose(GradVel,(0,2,1))
VortVec=np.array((W[:,2,1],W[:,0,2],W[:,1,0])).transpose()
DDT=np.einsum('abc,ace->abe', D, np.transpose(D,(0,2,1)))
Dnorm=np.sqrt(np.trace(DDT,axis1=1,axis2=2))
WWT=np.einsum('vux,vxz->vuz', W, np.transpose(W,(0,2,1)))
Wnorm=np.sqrt(np.trace(WWT,axis1=1,axis2=2))
DD=np.einsum('abc,ace->abe', D, D)
WW=np.einsum('vux,vxz->vuz', W, W)
ReStReStT=np.einsum('vux,vxz->vuz', ReStress, np.transpose(ReStress,(0,2,1)))
ReStnorm=np.sqrt(np.trace(ReStReStT,axis1=1,axis2=2))
VeldPdx=np.einsum('ab,ab->a', Vel, GradP)
dPdx2Vel2=np.einsum('xy,xv->x', np.power(GradP,2), np.power(Vel,2))
L_turb=np.power(K,1.5)/(Eps+SMALL) #integral lenght scale
Vort_jGrad_ij=np.einsum('vu,vyu->vyu', VortVec, GradVel)
Vort_jGrad_ijVort_kGradik=np.sum(np.triu(np.einsum('vuy,vuw->vuyw', Vort_jGrad_ij, Vort_jGrad_ij),0),axis=(1,2,3))
U_jGrad_ij=np.einsum('vu,vyu->vyu', Vel, GradVel)
U_iU_jGrad_ij=np.einsum('vu,vuy->v', Vel, U_jGrad_ij)
U_jGrad_ijU_kGradik=np.triu(np.einsum('vuy,vuw->vuyw', U_jGrad_ij, U_jGrad_ij),0)
UnUnU_jGrad_ijU_kGradik=np.einsum('vu,vyxw->v', (np.power(Vel,2)), U_jGrad_ijU_kGradik)
ReSt_ijDij=np.sum(np.triu(np.einsum('vux,vux->vux', ReStress, D)),axis=(1,2))

In [5]:
# Non-dimensionalized Q criterion
Q=((1/(pi))*np.arccos((np.power(Wnorm,2)-(np.power(Dnorm,2)))/(np.power(Wnorm,2)+(np.power(Dnorm,2)))))
# Turbulence intensity
kTurbNorm=K/(0.5*(np.einsum('ab,ab->a', Vel, Vel))+K+SMALL)
# Turbulence Reynolds number
LamVisc=0.001
Dens=1
dturb=1
ReTurb=np.minimum(((np.sqrt(K)*L_turb)/(50*LamVisc)),2.)
# Pressure gradient along streamline
PgradSL=VeldPdx/(np.sqrt(dPdx2Vel2)+np.abs(VeldPdx)+SMALL)
# Ratio of turbulent time scale to mean strain time scale
TurbTimeScale_StrainTimeScale=(Dnorm*K)/(Dnorm*K+Eps+SMALL)
# Viscosity ratio
ViscRatio=TurbVisc/(100*LamVisc+TurbVisc+SMALL)
# Ratio of pressure normal stresses to normal shear stresses
PressNormalStress_NormalShearStress=(np.sqrt(np.einsum('ab,ab->a', GradP, GradP)))/\
(np.sqrt(np.einsum('ab,ab->a', GradP, GradP))+0.5*Dens*(np.einsum('ab,ab->a', DivVel, DivVel))+SMALL)
# Vortex stretching
VortexStretch=np.sqrt(Vort_jGrad_ijVort_kGradik)/(np.sqrt(Vort_jGrad_ijVort_kGradik)+Dnorm+SMALL)
#Marker of Gorle et al deviation from parallel shear flow
GorlePar=np.abs(U_iU_jGrad_ij)/(np.sqrt(UnUnU_jGrad_ijU_kGradik)+np.abs(U_iU_jGrad_ij)+SMALL)
#Ratio of convection to production of k
#Ratio of total Reynolds stresses to normal Reynolds stresses
ReSt_Total_Normal=ReStnorm/(K+ReStnorm)
#Cubic eddy viscosity comparison

In [6]:
PositionLES=np.array((file[:,30],file[:,31],file[:,32])).transpose()
GradVelLES=np.transpose(np.array([[file[:,17],file[:,18],file[:,19]],
                                  [file[:,20],file[:,22],file[:,21]],
                                  [file[:,23],file[:,24],file[:,25]]]),(2,0,1))
ReStressLES=np.transpose(np.array([[file[:,33],file[:,34],file[:,35]],
                                   [file[:,34],file[:,36],file[:,37]],
                                   [file[:,35],file[:,37],file[:,38]]]),(2,0,1))
K_LES=0.5*np.einsum('aii', ReStressLES)
ReStressLES=ReStressLES-2/3*np.einsum('a,aij->aij',K_LES,np.tril(np.triu(np.ones((ReStressLES.shape[0],3,3)),0),0))
DLES=GradVelLES+np.transpose(GradVelLES,(0,2,1))
WLES=GradVel-np.transpose(GradVelLES,(0,2,1))
Anisotropy=(ReStressLES-2/3*np.einsum('a,aij->aij',K_LES,np.tril(np.triu(np.ones((ReStressLES.shape[0],3,3)),0),0)))
Anisotropy=np.einsum('a,aij->aij',np.power(K_LES,-1),Anisotropy)
AnisoAniso=np.einsum('abc,ace->abe', Anisotropy, Anisotropy)
ReStressD=-1*np.einsum('abc,abc->abc', GradVelLES, DLES)
KD_LES=(2/3)*np.einsum('a,aij->aij',K_LES,np.tril(np.triu(np.ones((DLES.shape[0],3,3)),0),0))
KD_LES=np.einsum('aij,aij->aij',KD_LES,D)

In [7]:
#Negative eddy viscosity error metric
nu_Turb_LES=ReStressD+KD_LES
nu_Turb_LES=np.einsum('aii', nu_Turb_LES)/(2*np.einsum('aij,aij->a', D,D)+SMALL)
condlistI=[nu_Turb_LES<0, nu_Turb_LES>=0]
choicelistI = [1, 0]
nu_Turb_LES_bin=np.select(condlistI, choicelistI)
#Anisotropy error metric
II_anisotensor=0.5*np.power(np.trace(Anisotropy,axis1=1,axis2=2),2)-np.trace(AnisoAniso,axis1=1,axis2=2)
condlistII=[-2*II_anisotensor>(1/6), -2*II_anisotensor<=(1/6)]
choicelistII = [1, 0]
II_anisotensor_bin=np.select(condlistII, choicelistII)

In [8]:
with open('malha1_re7500_dados_processados', 'w') as res:
    for l1,line in enumerate(file):
        res.write("{} {} {} {} {} {} {} {} {} {} {} {}\n".format(Q[l1],kTurbNorm[l1],ReTurb[l1],PgradSL[l1],\
                                                                 TurbTimeScale_StrainTimeScale[l1],ViscRatio[l1],\
                                                                 PressNormalStress_NormalShearStress[l1],VortexStretch[l1],\
                                                                 GorlePar[l1],ReSt_Total_Normal[l1],nu_Turb_LES_bin[l1],\
                                                                 II_anisotensor_bin[l1]))